In [1]:
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 843.4/843.4 kB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Created wheel for sagemaker: filename=sagemaker-2.167.0-py2.py3-none-any.whl size=1149213 sha256=47dbdd3bffd341cec5a5092ff1db7d84b29bfa99a865cf5289219b0d995f2104
  Stored in directory: /home/ec2-user/.cache/pip/wheels/3e/d3/39/c082337044a98f7bbf6cec664a9247d9cdce1ac5bd730ae2d8
Successfully built sagemaker
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled

In [2]:
import sagemaker
print(sagemaker.__version__)

2.167.0


In [3]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Matplotlib is building the font cache; this may take a moment.


Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [4]:
bucket_name = 'documentapplication' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

eu-north-1


In [5]:
s3 = boto3.resource('s3')

In [6]:
from botocore.exceptions import ClientError

# Check if the bucket already exists
try:
    s3.Bucket(bucket_name).load()
    print('Bucket already exists')
except ClientError as e:
    # If the bucket does not exist, create it with the specified region
    if e.response['Error']['Code'] == "404":
        try:
            s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
            print('Bucket created successfully')
        except Exception as e:
            print('S3 error: ', e)
    else:
        print('S3 error: ', e)

Bucket already exists


In [7]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [8]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [9]:
print("Total Rows: ", model_data.shape[0])
print("Total Columns: ", model_data.shape[1])

Total Rows:  41188
Total Columns:  61


In [10]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [11]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [12]:
boto3.Session().resource('s3').Bucket(bucket_name)

s3.Bucket(name='documentapplication')

In [13]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [14]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [15]:
s3_input_train


In [16]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.c5.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [17]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-21-08-49-51-653


2023-06-21 08:49:51 Starting - Starting the training job.........
2023-06-21 08:51:21 Downloading - Downloading input data......
2023-06-21 08:52:17 Training - Training image download completed. Training in progress.
2023-06-21 08:52:17 Uploading - Uploading generated training modelArguments: train
[2023-06-21:08:52:10:INFO] Running standalone xgboost training.
[2023-06-21:08:52:10:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-21:08:52:10:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 267.93mb
[2023-06-21:08:52:10:INFO] Determined delimiter of CSV input is ','
[08:52:10] S3DistributionType set as FullyReplicated
[08:52:10] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[08:52:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5
[0]#011train-error:0.100482
[08:52:10] src/tree/updater_prune.cc:74: tree prun

In [25]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.c5.xlarge', serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: xgboost-2023-06-21-09-17-45-389
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-06-21-09-17-45-389
INFO:sagemaker:Creating endpoint with name xgboost-2023-06-21-09-17-45-389


---!

In [22]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [23]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



In [43]:
model_data.iloc[0].values[:-2]

array([ 56,   1, 999,   0,   1,   0,   0,   0,   0,   1,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,
         0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   1,   0,   0,
         0,   1,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,
         1,   0,   0,   0,   0,   1,   0])

In [51]:
xgb_predictor.predict([model_data.iloc[0].values[:-2]]).decode('utf-8')

'0.04040025174617767'

In [50]:
xgb_predictor.predict([model_data.iloc[0].values[:-2]])

b'0.04040025174617767'

In [55]:
np.round(np.fromstring(xgb_predictor.predict([model_data.iloc[0].values[:-2]]), sep=','))[0] # and turn the prediction into an array

0.0

In [ ]:
56,   1, 999,   0,   1,   0,   0,   0,   0,   1,   0,   0,   0,0,   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   1,   0,   0,0,   1,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,1,   0,   0,   0,   0,   1,   0

In [ ]:
# xgb_predictor.delete_endpoint(delete_endpoint_config=True)

In [49]:
# curl -X POST https://o4x11sv8wk.execute-api.eu-north-1.amazonaws.com/test/demo -H 'Content-Type: application/json' -d '56,   1, 999,   0,   1,   0,   0,   0,   0,   1,   0,   0,   0,0,   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   1,   0,   0,0,   1,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,1,   0,   0,   0,   0,   1,   0'

In [47]:
import requests

In [57]:
# def input_fn(request_body, request_content_type):
#     print(request_body)
#     print(request_content_type)
#     if request_content_type == "text/csv":
#         request_body = request_body.strip()
#         try:
#             df = pd.read_csv(StringIO(request_body), header=None)
#             return df
        
#         except Exception as e:
#             print(e)
#     else:
#         return """Please use Content-Type = 'text/csv' and, send the request!!""" 